# NFL Data Models and Results

This notebook will go through the methodology of data collection, cleaning, and return a finished dataset of a narrowed-down analysis to passing/running plays in the 2018 NFL season. The data is obtained from the NFL's 2020 and 2021 Big Data Bowl:

*There is potential to combine the 2020 data bowl data, which contains similar info to 2021 data bowl data except about rushing plays 2017-2019. Combining these sources to produce a similar notebook to the original tendency analysis. Less data, but more information in our columns.*

The main focus of this analysis is to see how offensive / defensive personnel and the formation and defensive players on the field affect the decision to run or pass the ball. This can be a unique opportunity to utilize tracking / location data of players as well (which may be explored in a separate notebook).

Data bowls for reference:
- https://www.kaggle.com/c/nfl-big-data-bowl-2020: Forecast yardage gained on the run plays
- https://www.kaggle.com/c/nfl-big-data-bowl-2021/: Forecast yardage gained on pass plays
- https://www.kaggle.com/c/nfl-big-data-bowl-2022: Analyze special teams data
- https://github.com/nfl-football-ops/Big-Data-Bowl: Inaugural data bowl from 2019, useful R code on animation of tracking

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
import os
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [20]:
# Read in cleaned dataset for general models
bdb = pd.read_csv('bdb_2018_dummy.csv')

# Split response vs train data
bdb_y = np.where(bdb['Type']=='run', 1, 0) ## convert response variable to 1 for run, 0 for pass
bdb_x = bdb.drop(['Type'], axis=1)

# Models

In this section, we train 3 models on the train/test split: logistic regression model, random forest model, and gradient-boosted decision trees with XGBoost.

We do this on a train/test split of 75/25.

To do:
- K-fold cross-validation
- Tune hyperparameters...these are the absolute most bare versions of these models so far.
- Identify important features in prediction
- Set up a function to train these models at once

Other ideas:
- Segment the data by the position on the field (i.e. train a model for prediction when team is between their 0-20, then their own 20-50, then opposing 50-80, then redzone on opposing 20)
- Train models by team on offense
- Train models by down #

In [21]:
# Create train / test split
x_train, x_test, y_train, y_test = train_test_split(bdb_x, bdb_y, test_size=0.25, random_state=2022)
print('Training Rows: {}\nTest Rows: {}'.format(x_train.shape[0], x_test.shape[0]))

Training Rows: 21439
Test Rows: 7147


In [22]:
## Train Logistic Regression model
lr = LogisticRegression()
lr.fit(x_train, y_train)

C:\Users\tshah\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [47]:
## Train RF model
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

RandomForestClassifier()

In [54]:
## Train XGBoost model
xgc = xgb.XGBClassifier()
xgc.fit(x_train, y_train)

[08:02:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Results

In [42]:
1- (sum(bdb_y) / len(bdb_y)) ## Benchmark: We can get 62.16% success rate just guessing pass every play

0.6215979850276359

In [50]:
## Predict logistic regression
y_pred = lr.predict(x_test)
print(lr.score(x_test, y_test))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7412900517699734
[[3668  767]
 [1082 1630]]
              precision    recall  f1-score   support

           0       0.77      0.83      0.80      4435
           1       0.68      0.60      0.64      2712

    accuracy                           0.74      7147
   macro avg       0.73      0.71      0.72      7147
weighted avg       0.74      0.74      0.74      7147



In [49]:
## Predict RF
y_pred = rf.predict(x_test)
print(rf.score(x_test, y_test))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7464670491115153
[[3593  842]
 [ 970 1742]]
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      4435
           1       0.67      0.64      0.66      2712

    accuracy                           0.75      7147
   macro avg       0.73      0.73      0.73      7147
weighted avg       0.74      0.75      0.75      7147



In [55]:
## Predict XGB
y_pred = xgc.predict(x_test)
print(xgc.score(x_test, y_test))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7503847768294389
[[3558  877]
 [ 907 1805]]
              precision    recall  f1-score   support

           0       0.80      0.80      0.80      4435
           1       0.67      0.67      0.67      2712

    accuracy                           0.75      7147
   macro avg       0.73      0.73      0.73      7147
weighted avg       0.75      0.75      0.75      7147

